In [10]:
import yfinance as yf
import pandas as pd
from datetime import date

In [11]:
## I acquire stock list in thailand ###
ex = pd.read_excel('listedCompanies_th_TH.xlsx', sheet_name='listedCompanies_th_TH',skiprows=1)
df = pd.DataFrame(ex['หลักทรัพย์'].astype('str') + '.bk')
df = df.rename(columns={'หลักทรัพย์' : 'ticker'})
df

,ticker
0,24CS.bk
1,2S.bk
2,3BBIF.bk
3,3K-BAT.bk
4,A.bk
...,...
916,YONG.bk
917,YUASA.bk
918,ZAA.bk
919,ZEN.bk


### Acquire informations from yfinance function >> .info

In [12]:
# 1st pulling date for this data #
df['date_pulling'] = date.today()
df

,ticker,date_pulling
0,24CS.bk,2024-08-08
1,2S.bk,2024-08-08
2,3BBIF.bk,2024-08-08
3,3K-BAT.bk,2024-08-08
4,A.bk,2024-08-08
...,...,...
916,YONG.bk,2024-08-08
917,YUASA.bk,2024-08-08
918,ZAA.bk,2024-08-08
919,ZEN.bk,2024-08-08


In [13]:
# 2nd acquiring information from .info #
info_attribute_list = [
    'industry',
    'sector',
    'marketCap',
    'sharesOutstanding',
    'currentPrice',
    'fiveYearAvgDividendYield'

]

info_attribute_list_buffer = []
for i in range(len(df)):
    info_attribute_list_buffer_c = []
    info_attribute_list_tickerinfo = yf.Ticker(df['ticker'][i])
    for j in range(len(info_attribute_list)):
        try:
            info_attribute_list_buffer_c.append(info_attribute_list_tickerinfo.info[info_attribute_list[j]])
        except:
            info_attribute_list_buffer_c.append('')
    info_attribute_list_buffer.append(info_attribute_list_buffer_c)

df = df.join(pd.DataFrame(info_attribute_list_buffer, columns=info_attribute_list))

In [16]:
df.loc[df['ticker'] == 'SAUCE.bk']

,ticker,date_pulling,industry,sector,marketCap,sharesOutstanding,currentPrice,fiveYearAvgDividendYield
631,SAUCE.bk,2024-08-08,Packaged Foods,Consumer Defensive,14759999488,360000000,41.0,4.53


In [ ]:
# 3rd acquiring information from .quaterly_financials #

